# Libraries

In [1]:
import numpy as np

# Inputs

In [2]:
# Define Big M
M = 'M'

# Minimize or Maximize Z
is_min = True

# Objective Function Z = c_T * x
c_T = [2, 4, 0, 0, M, M]

# A * x = b
A = [
    [1, 2, -1, 0, 1, 0],
    [-1, 2, 0, -1, 0, 1]
]
b_T = [4, 6]

# Find number of rows and columns
num_rows = len(A)
num_cols = len(A[0])

In [3]:
# Define Big M
M = 'M'

# Minimize or Maximize Z
is_min = True

# Objective Function Z = c_T * x
c_T = [-3/4, 20, -1/2, 6, 0, 0, 0]

# A * x = b
A = [
    [1/4, -8, -1, 9, 1, 0, 0],
    [1/2, -12, -1/2, 3, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 1]
]
b_T = [0, 0, 1]

# Find number of rows and columns
num_rows = len(A)
num_cols = len(A[0])

# Verify Problem Integrity

In [4]:
assert len(c_T) == num_cols
assert len(b_T) == num_rows

# Transform to NumPy

In [5]:
c_TR = np.array([c if c!=M else 0 for c in c_T])
c_TM = np.array([1 if c==M else 0 for c in c_T])
b_T = np.array(b_T)
A = np.array(A)

# Find Initial Solution

In [6]:
basis = [-1] * num_rows

for j in range(num_cols):
    column = A[:, j]
    
    # A column is part of the identity matrix if:
    # - It has exactly one '1'
    # - The rest of the elements are '0'
    if np.sum(column == 1) == 1 and np.sum(column == 0) == (num_rows - 1):
        row_idx = np.where(column == 1)[0][0]
        
        # If this row doesn't have a basic variable assigned yet, assign it
        if basis[row_idx] == -1:
            basis[row_idx] = j

# 2. Construct the initial Basic Feasible Solution (BFS) vector x
x = np.zeros(num_cols)
for i, col_idx in enumerate(basis):
    if col_idx != -1:
        x[col_idx] = b_T[i]
    else:
        print(f"Warning: No identity column found for row {i}. Check if artificial variables are needed.")

# Convert basis to a numpy array for easier indexing later
basis = np.array(basis)

print("Initial Basis (column indices):", basis)
print("Initial x:", x)

Initial Basis (column indices): [4 5 6]
Initial x: [0. 0. 0. 0. 0. 0. 1.]


# Define Class Tableau

In [7]:
import numpy as np

class SimplexTableau:
    def __init__(self, c_TR, c_TM, b_T, A, x, basis, is_minimize=True, pivot_rule='bland'):
        self.c_TR = np.array(c_TR, dtype=float)
        self.c_TM = np.array(c_TM, dtype=float)
        self.b_T = np.array(b_T, dtype=float)
        self.A = np.array(A, dtype=float)
        self.x = np.array(x, dtype=float)
        self.basis = np.array(basis, dtype=int)
        self.is_minimize = is_minimize
        self.pivot_rule = pivot_rule
        
        self.num_rows, self.num_cols = self.A.shape
        
        # History to detect cycling (stores tuples of the basis)
        self.history = set()
        self._record_state()

        self.r_R = np.zeros(self.num_cols)
        self.r_M = np.zeros(self.num_cols)
        self.reduce_costs()

    def _record_state(self):
        """Saves the current basis to history. Returns False if already seen."""
        state = tuple(self.basis.tolist())
        if state in self.history:
            return False
        self.history.add(state)
        return True

    def reduce_costs(self):
        cb_R = self.c_TR[self.basis]
        cb_M = self.c_TM[self.basis]
        self.r_R = self.c_TR - np.dot(cb_R, self.A)
        self.r_M = self.c_TM - np.dot(cb_M, self.A)

    def check_optimal(self):
        multiplier = 1 if self.is_minimize else -1
        for j in range(self.num_cols):
            if multiplier * self.r_M[j] < -1e-9:
                return False
            if abs(self.r_M[j]) < 1e-9 and multiplier * self.r_R[j] < -1e-9:
                return False
        return True

    def _is_negative(self, val):
        return val < -1e-9

    def get_entering_variable(self):
        multiplier = 1 if self.is_minimize else -1
        
        # Big-M part
        candidates_M = []
        for j in range(self.num_cols):
            val = multiplier * self.r_M[j]
            if self._is_negative(val):
                candidates_M.append((j, val))

        if candidates_M:
            if self.pivot_rule == "bland":
                return min(j for j, _ in candidates_M)
            return min(candidates_M, key=lambda x: x[1])[0]

        # Real part
        candidates_R = []
        for j in range(self.num_cols):
            if abs(self.r_M[j]) < 1e-9:
                val = multiplier * self.r_R[j]
                if self._is_negative(val):
                    candidates_R.append((j, val))

        if candidates_R:
            if self.pivot_rule == "bland":
                return min(j for j, _ in candidates_R)
            return min(candidates_R, key=lambda x: x[1])[0]

        return -1

    def iterate(self):
        if self.check_optimal():
            return "OPTIMAL"

        pivot_col = self.get_entering_variable()
        if pivot_col == -1: return "OPTIMAL"

        # Ratio test
        ratios = []
        for i in range(self.num_rows):
            if self.A[i, pivot_col] > 1e-9:
                ratios.append(self.b_T[i] / self.A[i, pivot_col])
            else:
                ratios.append(np.inf)
        
        min_ratio = np.min(ratios)
        if min_ratio == np.inf:
            raise ValueError("Problem is unbounded.")

        # Check for degeneracy: if multiple rows have the same minimum ratio
        # or if the minimum ratio is 0.
        if min_ratio < 1e-9:
            print(f"⚠️ Degenerate step detected (Ratio = 0 at pivot column {pivot_col})")

        pivot_row = np.argmin(ratios)
        
        # Perform Pivoting
        pivot_val = self.A[pivot_row, pivot_col]
        self.A[pivot_row, :] /= pivot_val
        self.b_T[pivot_row] /= pivot_val
        
        for i in range(self.num_rows):
            if i != pivot_row:
                factor = self.A[i, pivot_col]
                self.A[i, :] -= factor * self.A[pivot_row, :]
                self.b_T[i] -= factor * self.b_T[pivot_row]
        
        # Update Basis
        self.basis[pivot_row] = pivot_col
        
        # Check for Cycling
        if not self._record_state():
            raise RuntimeError("Cycling detected! The algorithm has returned to a previous basis.")
            
        self.reduce_costs()
        return "CONTINUE"

    def get_current_solution(self):
        sol = np.zeros(self.num_cols)
        for i, col_idx in enumerate(self.basis):
            sol[col_idx] = self.b_T[i]
        return sol

In [8]:
# --- Main Execution Loop ---

# 1. Initialize the Tableau
# Assuming c_TR, c_TM, b_T, A, x, basis, and is_minimize are defined in previous cells
tableau = SimplexTableau(c_TR, c_TM, b_T, A, x, basis, is_minimize=True, pivot_rule='max')

max_iterations = 100
iteration_count = 0
success = False

print(f"{'Iter':<5} | {'Objective (M-part)':<20} | {'Objective (Real-part)':<20}")
print("-" * 55)

while iteration_count < max_iterations:
    # Calculate current objective values
    current_x = tableau.get_current_solution()
    obj_R = np.dot(tableau.c_TR, current_x)
    obj_M = np.dot(tableau.c_TM, current_x)
    
    print(f"{iteration_count:<5} | {obj_M:<20.4f} | {obj_R:<20.4f}")

    if tableau.check_optimal():
        print("-" * 55)
        print("✅ Optimal solution found!")
        success = True
        break
    
    try:
        status = tableau.iterate()
        if status == "OPTIMAL":
            break
    except RuntimeError as cycling_error:
        print(f"🛑 Error: {cycling_error}")
        break
        
    iteration_count += 1

if not success and iteration_count == max_iterations:
    print("⚠️ Reached maximum iterations. The problem might be cycling or degenerate.")

# --- Final Results ---
if success:
    final_x = tableau.get_current_solution()
    final_obj_R = np.dot(tableau.c_TR, final_x)
    final_obj_M = np.dot(tableau.c_TM, final_x)
    
    print("\nFinal Results:")
    print(f"Optimal x: {final_x}")
    
    # In Big-M, if the M-part of the objective is not zero, the original problem is infeasible
    if abs(final_obj_M) > 1e-6:
        print("Status: Infeasible (Artificial variables remain in the basis with non-zero value)")
    else:
        print(f"Status: Feasible and Optimal")
        print(f"Optimal Objective Value: {final_obj_R:.4f}")

Iter  | Objective (M-part)   | Objective (Real-part)
-------------------------------------------------------
0     | 0.0000               | 0.0000              
⚠️ Degenerate step detected (Ratio = 0 at pivot column 0)
1     | 0.0000               | 0.0000              
⚠️ Degenerate step detected (Ratio = 0 at pivot column 1)
2     | 0.0000               | 0.0000              
⚠️ Degenerate step detected (Ratio = 0 at pivot column 2)
3     | 0.0000               | 0.0000              
⚠️ Degenerate step detected (Ratio = 0 at pivot column 3)
4     | 0.0000               | 0.0000              
⚠️ Degenerate step detected (Ratio = 0 at pivot column 4)
5     | 0.0000               | 0.0000              
⚠️ Degenerate step detected (Ratio = 0 at pivot column 5)
🛑 Error: Cycling detected! The algorithm has returned to a previous basis.
